In [4]:
!pwd

/media/yamba/193b2d55-8f5d-44c9-8078-07220e0aecba/yamba/Documents/CovidMDP/notebooks


In [1]:
import sys 
import os
sys.path.append(os.path.abspath("/media/yamba/193b2d55-8f5d-44c9-8078-07220e0aecba/yamba/Documents/CovidMDP/src/"))

In [2]:
os.chdir("../src/models")

In [3]:
from numpy.random import default_rng
import numpy as np
from tqdm import tqdm

In [4]:
from models.simulate_pandemic import init_infection, spread_infection, lambda_leak_expose, update_population

In [5]:
from actions import city_restrictions

In [6]:
policy='Unrestricted'
days=350
step_size=7
disable_tqdm=False
seed=None

In [7]:
prhome = 0.06
p_r = {
    'home'    :  prhome,
    'neighbor':  .1*prhome,
    'work'    :  .1*prhome,
    'school'  :  .15*prhome,
}

g_pickle = '../../data/processed/SP_multiGraph_Job_Edu_Level.gpickle'

In [8]:
rng = default_rng(seed)
pop_matrix, adj_list = init_infection(g_pickle)
data = []
total_steps = int(np.ceil(days/step_size))

if isinstance(policy, str):
    policy = total_steps * [policy]

if len(policy) < total_steps:
    raise ValueError(f'len of policy should be at least {total_steps}')

for day in tqdm(range(1, days+1), disable=disable_tqdm):
    # if less than 90% already recovered, break simulation
    if (pop_matrix[pop_matrix[:, 1] == -1].shape[0] > pop_matrix.shape[0]*.9):
        break

    if day % step_size == 1:          
        current_step = int(day/step_size)
        restrictions = city_restrictions[policy[current_step]]

    pop_matrix = spread_infection(pop_matrix, adj_list, restrictions, day, rng, p_r)
    pop_matrix = lambda_leak_expose(pop_matrix, day)
    pop_matrix = update_population(pop_matrix)

    data.append(pop_matrix[:, 0:2])

  1%|          | 2/350 [00:02<07:11,  1.24s/it]


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'adj_list': Cannot determine Numba type of <class 'dict'>

File "simulate_pandemic.py", line 259:
    def get_exposed(p_r, currently_infected, restrictions):
        <source elided>
        for relation, prob in p_r.items():
            pairs = [contact 
            ^


In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame([pd.Series(d[:, 1]).value_counts() for d in data])

In [ ]:
import networkx as nx

In [ ]:
for x in p_r.items():
    print(x)

In [ ]:
G = nx.read_gpickle(g_pickle)


In [ ]:
unique_types = np.unique([data['edge_type'] for x,y, data in G.edges(data=True)])

edge_list = {
    edge_type: np.array([(u,v) for u,v,e in G.edges(data=True) if e['edge_type'] == edge_type])
                            for edge_type in unique_types
}

In [ ]:
currently_infected = rng.choice(G.nodes, size=10000)

In [ ]:
%%timeit
exposed = []

def filter_contacts(rel, prob, restrictions, currently_infected, edge_list):
    ed = edge_list[rel]
    contacts = ed[np.isin(ed[:,0], currently_infected)][:, 1]
    mask = rng.random(size=len(contacts)) < prob * (1-restrictions[rel])

    return contacts[mask]

args = [restrictions, currently_infected, edge_list]
exposed = list(map(lambda x: filter_contacts(x[0], x[1], *args), [x for x in p_r.items()]))
exposed = np.unique(np.concatenate(exposed)) 

In [ ]:
%%timeit
exposed = []

def filter_contacts(rel, prob, restrictions, currently_infected, edge_list):
    ed = edge_list[rel]
    contacts = ed[np.isin(ed[:,0], currently_infected)][:, 1]
    mask = rng.random(size=len(contacts)) < prob * (1-restrictions[rel])

    return contacts[mask]

args = [restrictions, currently_infected, edge_list]
exposed = list(map(lambda x: filter_contacts(x[0], x[1], *args), [x for x in p_r.items()]))
exposed = np.unique(np.concatenate(exposed)) 

In [ ]:
%%timeit

exposed = []
for rel, prob in p_r.items():
    ed = edge_list[rel]
    contacts = ed[np.isin(ed[:,0], currently_infected)][:, 1]
    mask = rng.random(size=len(contacts)) < prob * (1-restrictions[rel])
    exposed += [contacts[mask]]

exposed = np.unique(np.concatenate(exposed))